In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [148]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'

In [149]:
stats = pd.read_csv(path + '/all_files_stats.csv')
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
files

,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
0,name/ Product_stats.csv,name,string,0.994361,Product_stats.csv,432728.0,product_name
1,offers/ Product_stats.csv,offers,string,0.964249,Product_stats.csv,419624.0,offer
2,brand/ Product_stats.csv,brand,string,0.403728,Product_stats.csv,175695.0,product_name
3,headline/ CreativeWork_stats.csv,headline,string,0.939871,CreativeWork_stats.csv,76451.0,headline
4,author/ CreativeWork_stats.csv,author,string,0.776499,CreativeWork_stats.csv,63162.0,person_name
...,...,...,...,...,...,...,...
202,numtracks/ MusicAlbum_stats.csv,numtracks,int,0.415808,MusicAlbum_stats.csv,121.0,number
203,menu/ Restaurant_stats.csv,menu,string,0.104581,Restaurant_stats.csv,121.0,text_description
204,contactpoint/ Person_stats.csv,contactpoint,string,0.009353,Person_stats.csv,120.0,text_description
205,email/ Hotel_stats.csv,email,string,0.107664,Hotel_stats.csv,118.0,email


In [150]:
listcol= files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
listcol.filename = listcol.filename.str.split('_', expand=True)[0]
listcol.index = listcol['filename']
listcol

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [183]:
listcolt = listcol[listcol['filename']=='LocalBusiness']
listcolt

,filename,listofcolumns
filename,,
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."


In [168]:
def process(profiles):
    #print(filename)
    df = pd.read_json(os.path.join(inputdir+profiles), compression='gzip', lines=True)
    #if len(df)<=50:
        #print(len(df))
    #    return
    setfile = set(df.columns)
    length = len(setfile & setcol)
    #print(length)
    if (length >= 2): #& (length<= 12):
        df.to_json(os.path.join(outputdir+profiles), compression='gzip', orient='records', lines=True)

In [169]:
if __name__ == '__main__':
    for index in listcolt.filename:
        print(index)
        setcol = set(listcolt.loc[index]['listofcolumns'])
        print(setcol)
        inputdir = '../../src/data/data/'+index+'/All/'
        outputdir = '../../src/data/schemafiltereddata/'+index+'/'
        if not os.path.exists(outputdir):
            os.makedirs(outputdir)
        p = Pool(100)
        for _ in tqdm(p.imap_unordered(process, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]), total=len([file for file in os.listdir(inputdir) if file.endswith('.json.gz')])):
            pass
        p.map(process, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')])
        p.close()
        print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
        p.join()
        

Person
{'affiliation', 'birthdate', 'givenname', 'jobtitle', 'gender', 'deathdate', 'mainentityofpage', 'address', 'familyname', 'height', 'sameas', 'contactpoint', 'weight', 'birthplace', 'worksfor', 'telephone', 'nationality', 'knowslanguage', 'homelocation', 'worklocation', 'faxnumber', 'makesoffer', 'additionalname', 'memberof', 'email'}


100%|██████████| 12830/12830 [00:23<00:00, 552.85it/s]


4436


In [156]:
num_tables=[]
for index in listcolt.filename:
    print(index)
    outputdir = '../../src/data/schemafiltereddata/'+index+'/'
    print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
    num_tables.append(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
len(num_tables)

Person
2814


1

In [188]:
listcolt = pd.read_csv('../../src/data/data/CSV_files/stats_filtered_and_cleaned_tables.csv')
listcolt

,filename,filename.1,listofcolumns,numtables
0,Book,Book,"['name', 'author', 'isbn', 'publisher', 'offer...",978
1,CreativeWork,CreativeWork,"['headline', 'author', 'datepublished', 'datem...",7933
2,Event,Event,"['name', 'enddate', 'location', 'performer', '...",6798
3,Hotel,Hotel,"['name', 'address', 'telephone', 'pricerange',...",1001
4,LocalBusiness,LocalBusiness,"['name', 'address', 'telephone', 'geo', 'price...",4252
5,MusicAlbum,MusicAlbum,"['name', 'byartist', 'track', 'numtracks']",146
6,MusicRecording,MusicRecording,"['name', 'duration', 'byartist', 'inalbum', 'd...",12661
7,Person,Person,"['mainentityofpage', 'jobtitle', 'birthdate', ...",1600
8,Place,Place,"['name', 'geo', 'telephone', 'sameas', 'openin...",1305
9,Product,Product,"['name', 'offers', 'brand', 'aggregaterating',...",3168


In [184]:
column_count = []
path='../../src/data/schemafiltereddata/'+'LocalBusiness'+'/'
files = [file for file in os.listdir(path) if file.endswith('.json.gz')]
for file in files:
    # read file
    df = pd.read_json(os.path.join(path, '{}'.format(file)), compression='gzip', lines=True)
    list_col = list(df.columns)
    column_count.extend(list_col)

In [189]:
listcolt = listcolt.set_index('filename')
listcolt

,filename.1,listofcolumns,numtables
filename,,,
Book,Book,"['name', 'author', 'isbn', 'publisher', 'offer...",978
CreativeWork,CreativeWork,"['headline', 'author', 'datepublished', 'datem...",7933
Event,Event,"['name', 'enddate', 'location', 'performer', '...",6798
Hotel,Hotel,"['name', 'address', 'telephone', 'pricerange',...",1001
LocalBusiness,LocalBusiness,"['name', 'address', 'telephone', 'geo', 'price...",4252
MusicAlbum,MusicAlbum,"['name', 'byartist', 'track', 'numtracks']",146
MusicRecording,MusicRecording,"['name', 'duration', 'byartist', 'inalbum', 'd...",12661
Person,Person,"['mainentityofpage', 'jobtitle', 'birthdate', ...",1600
Place,Place,"['name', 'geo', 'telephone', 'sameas', 'openin...",1305


In [191]:
import ast
LocalB_list = listcolt.loc['LocalBusiness']['listofcolumns']
LocalB_list = ast.literal_eval(LocalB_list)
LocalB_list

['name',
 'address',
 'telephone',
 'geo',
 'pricerange',
 'aggregaterating',
 'email',
 'sameas',
 'faxnumber',
 'openinghours',
 'openinghoursspecification',
 'review',
 'contactpoint',
 'legalname',
 'areaserved',
 'location',
 'addresslocality',
 'addressregion',
 'streetaddress',
 'postalcode',
 'paymentaccepted',
 'founder',
 'citystatezip',
 'owns',
 'hasmap']

In [193]:
colLocalB = pd.DataFrame()
colLocalB['colcount_LocalB'] = column_count
colLocalB = colLocalB.groupby('colcount_LocalB').agg({'colcount_LocalB':'count'})
colLocalB = colLocalB.rename(columns={'colcount_LocalB':'num_col'}).sort_values(by='num_col').reset_index()
colLocalB = colLocalB[colLocalB['colcount_LocalB'].isin(LocalB_list)]
colLocalB

,colcount_LocalB,num_col
112,owns,40
116,founder,57
120,location,118
121,hasmap,120
123,paymentaccepted,150
125,citystatezip,198
126,streetaddress,231
127,addressregion,231
128,postalcode,232
129,addresslocality,236


In [194]:
display(colbook)
colbook.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsbook.csv')
colmusica.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsMusicAlbum.csv')
coltv.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsTVEpisode.csv')
colRestaurant.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsRestaurant.csv')
colPerson.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsPerson.csv')
colHotel.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsHotel.csv')
colPlace.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsPlace.csv')
colProduct.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsProduct.csv')

display(colmusica)
display(coltv)
display(colRestaurant)
display(colHotel)
display(colPerson)
display(colPlace)
display(colProduct)

,colcount_book,num_col
98,aggregaterating,188
99,bookedition,191
100,genre,244
101,inlanguage,290
102,bookformat,380
103,offers,568
104,numberofpages,608
106,datepublished,657
107,publisher,702
108,isbn,758


,colcount_musica,num_col
50,numtracks,121
51,track,150
52,byartist,179
53,name,240


,colcount_tv,num_col
52,episodenumber,111
55,partofseries,160
58,name,210


,colcount_Restaurant,num_col
54,menu,115
55,openinghours,127
57,openinghoursspecification,295
58,acceptsreservations,303
59,hasmenu,311
60,aggregaterating,356
62,geo,540
63,pricerange,614
64,servescuisine,653
65,telephone,699


,colcount_Hotel,num_col
79,email,118
82,starrating,226
83,aggregaterating,445
84,geo,476
85,pricerange,493
87,telephone,697
88,address,945
89,name,990


,colcount_Person,num_col
124,mainentityofpage,100
125,contactpoint,119
126,worklocation,121
127,memberof,125
128,faxnumber,144
129,birthplace,156
130,sameas,158
132,makesoffer,274
133,knowslanguage,303
135,homelocation,386


,colcount_Place,num_col
31,openinghoursspecification,120
34,sameas,326
35,telephone,1167
36,geo,1209
38,name,1276


,colcount_Product,num_col
48,breadcrumb,1
104,reviews,4
110,condition,5
115,itemlistelement,7
116,datepublished,7
118,title,7
120,author,10
125,geo,21
126,address,22
136,availability,46
